In [1]:
def find_longest_string(list_of_strings):
    longest_string = None
    longest_string_len = 0     
    for s in list_of_strings:
        if len(s) > longest_string_len:
            longest_string_len = len(s)
            longest_string = s    
    return longest_string

In [2]:
list_of_strings = ['abc', 'python', 'dima']
%time max_length = print(find_longest_string(list_of_strings))

python
CPU times: user 284 µs, sys: 28 µs, total: 312 µs
Wall time: 255 µs


In [3]:
large_list_of_strings = list_of_strings*1000
%time print(find_longest_string(large_list_of_strings))

python
CPU times: user 876 µs, sys: 85 µs, total: 961 µs
Wall time: 916 µs


In [ ]:
large_list_of_strings = list_of_strings*100000000
%time max_length = max(large_list_of_strings, key=len)

In [4]:
%%time

# step 1:
list_of_string_lens = (len(s) for s in large_list_of_strings)
list_of_string_lens = zip(large_list_of_strings, list_of_string_lens)
#step 2:
max_len = max(list_of_string_lens, key=lambda t: t[1])
print(max_len)

('python', 6)
CPU times: user 2.86 ms, sys: 0 ns, total: 2.86 ms
Wall time: 2.74 ms


In [5]:
mapper = len
def reducer(p, c):
    if p[1] > c[1]:
        return p
    return c

In [6]:
%%time

from functools import reduce
#step 1
mapped = map(mapper, large_list_of_strings)
mapped = zip(large_list_of_strings, mapped)
#step 2:
reduced = reduce(reducer, mapped)
print(reduced)

('python', 6)
CPU times: user 1.54 ms, sys: 147 µs, total: 1.69 ms
Wall time: 1.54 ms


In [7]:
## !pip install chunkify #Not working
def chunkify(_list, number_of_chunks=8):
    chunk_size = len(_list) // number_of_chunks
    # return list
    # return [ _list[i*chunk_size:(i+1)*chunk_size] for i in range(number_of_chunks) ]
    # return iterator
    for i in range(number_of_chunks):
        yield _list[i*chunk_size:(i+1)*chunk_size]
    raise StopIteration
## move to here, since local within %time got removed
def chunks_mapper(chunk):
    mapped_chunk = map(mapper, chunk) 
    mapped_chunk = zip(chunk, mapped_chunk)
    return reduce(reducer, mapped_chunk)

In [8]:
from functools import reduce
data_chunks = chunkify(large_list_of_strings, number_of_chunks=36)
#step 1:
reduced_all = []
for chunk in data_chunks:
    mapped_chunk = map(mapper, chunk)
    mapped_chunk = zip(chunk, mapped_chunk)
    
    reduced_chunk = reduce(reducer, mapped_chunk)
    reduced_all.append(reduced_chunk)
    
#step 2:
reduced = reduce(reducer, reduced_all)
print(reduced)

('python', 6)


In [10]:
%%time

data_chunks = chunkify(large_list_of_strings, number_of_chunks=30)
#step 1:
mapped = map(chunks_mapper, data_chunks)
#step 2:
reduced = reduce(reducer, mapped)
print(reduced)

('python', 6)
CPU times: user 1.22 ms, sys: 130 µs, total: 1.35 ms
Wall time: 1.29 ms


In [9]:
%%time

from multiprocessing import Pool
pool = Pool(8)
data_chunks = chunkify(large_list_of_strings, number_of_chunks=8)
#step 1:
mapped = pool.map(chunks_mapper, data_chunks)
#step 2:
reduced = reduce(reducer, mapped)
print(reduced)

('python', 6)
CPU times: user 26.9 ms, sys: 24.7 ms, total: 51.6 ms
Wall time: 192 ms
